<a href="https://colab.research.google.com/github/110805/Retinopathy_detection/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/110805/Retinopathy_detection.git
%cd Retinopathy_detection/

Cloning into 'Retinopathy_detection'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 35144 (delta 3), reused 2 (delta 1), pack-reused 35135
Receiving objects: 100% (35144/35144), 556.71 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (35133/35133), done.
/content/Retinopathy_detection


In [0]:
import pandas as pd
from torch.utils import data
import numpy as np
import torch
import cv2

def getData(mode):
    if mode == 'train':
        img = pd.read_csv('train_img.csv')
        label = pd.read_csv('train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv('test_img.csv')
        label = pd.read_csv('test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)


class RetinopathyLoader(data.Dataset):
    def __init__(self, root, mode):
        """
        Args:
            root (string): Root path of the dataset.
            mode : Indicate procedure status(training or testing)

            self.img_name (string list): String list that store all image names.
            self.label (int or float list): Numerical list that store all ground truth label values.
        """
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))

    def __len__(self):
        """'return the size of dataset"""
        return len(self.img_name)

    def __getitem__(self, index):
        """something you should implement here"""

        """
           step1. Get the image path from 'self.img_name' and load it.
                  hint : path = root + self.img_name[index] + '.jpeg'
           
           step2. Get the ground truth label from self.label
                     
           step3. Transform the .jpeg rgb images during the training phase, such as resizing, random flipping, 
                  rotation, cropping, normalization etc. But at the beginning, I suggest you follow the hints. 
                       
                  In the testing phase, if you have a normalization process during the training phase, you only need 
                  to normalize the data. 
                  
                  hints : Convert the pixel value to [0, 1]
                          Transpose the image shape from [H, W, C] to [C, H, W]
                         
            step4. Return processed image and label
        """
        img_path = self.root + self.img_name[index] + '.jpeg'
        img = cv2.imread(img_path)

        label = self.label[index] 

        normalizedImg = np.zeros((512, 512, 3))
        normalizedImg = cv2.normalize(img,  normalizedImg, 0, 1, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        normalizedImg = np.reshape(normalizedImg, (3, 512, 512))
        normalizedImg = torch.from_numpy(normalizedImg)
        label = torch.from_numpy(np.array([label]))

        return (normalizedImg, label)

train_data = RetinopathyLoader(root='/content/Retinopathy_detection/data/', mode='train')
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4)
for i, j in train_loader:
    print(i.shape)
    print(j.shape)


In [25]:
from dataloader import RetinopathyLoader
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models
import matplotlib.pyplot as plt
import time
import copy

# Hyperparameter setting
batch_size = 4
learning_rate = 1e-3
epochs_18 = 10
epochs_50 = 5
momentum = 0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()

train_data = RetinopathyLoader(root='/content/Retinopathy_detection/data/', mode='train')
test_data = RetinopathyLoader(root='/content/Retinopathy_detection/data/', mode='test')
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.size())
print(labels.size())

class ResNet(nn.Module):
    def __init__(self, pretrained=True, res_type=50):
        super(ResNet, self).__init__()

        self.classify = nn.Linear(2048, 5)

        pretrained_model = torchvision.models.__dict__['resnet{}'.format(res_type)](pretrained=pretrained)
        self.conv1 = pretrained_model._modules['conv1']
        self.bn1 = pretrained_model._modules['bn1']
        self.relu = pretrained_model._modules['relu']
        self.maxpool = pretrained_model._modules['maxpool']

        self.layer1 = pretrained_model._modules['layer1']
        self.layer2 = pretrained_model._modules['layer2']
        self.layer3 = pretrained_model._modules['layer3']
        self.layer4 = pretrained_model._modules['layer4']

        self.avgpool = nn.AdaptiveAvgPool2d(1)

        del pretrained_model

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classify(x)

        return x

models = [ResNet(pretrained=False, res_type=18), ResNet(pretrained=True, res_type=18), ResNet(pretrained=False, res_type=50), ResNet(pretrained=True, res_type=50)]
optimizer = optim.SGD(models[0].parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
device = torch.device('cuda')

def train_model(model, optimizer, num_epochs):
    since = time.time()

    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()
        correct = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            print(type(labels))
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1) # the second return of max is the return of argmax
            loss.backward()
            optimizer.step()

            correct += torch.sum(preds == labels.data)
            
        epoch_acc = correct.double() / 28099
            
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())       

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model = train_model(models[0], optimizer, epochs_18)

> Found 28099 images...
> Found 7025 images...


TypeError: ignored